In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from typing import Generator, Any
import pandas as pd
import json
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os
from rich import print as rprint
from mbay_nmt.utils import domain as d
from mbay_nmt.utils.models import new_object_id
from datasets import load_dataset, Dataset, DatasetDict
from rich import print as rprint

load_dotenv()

True

In [43]:
# from huggingface_hub import notebook_login

# notebook_login()

In [44]:
uri = os.environ["MONGODB_URI"]

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi("1"))

# Send a ping to confirm a successful connection
try:
    client.admin.command("ping")
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [45]:
entries = [
    d.Entry(**entry)
    for entry in client.get_database("dictionary").get_collection("entries-prod").find()
]

In [56]:
entry_map = {entry.id: entry for entry in entries}

In [46]:
entries[0]

Entry(id=ObjectId('64eca312f6197fd20d762cf5'), created_at=datetime.datetime(2023, 9, 9, 12, 27, 55, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2023, 9, 9, 12, 27, 55, tzinfo=TzInfo(UTC)), headword='àlmbétɨ̀, àlmétɨ̀', part_of_speech='NI', sound_filename='NewExpSS2215.mp3', french=Translation(translation='match', key='m'), english=Translation(translation='match.', key='m'), related_word=None, grammatical_note=None, examples=[Example(id=ObjectId('64eca312f6197fd20d76096f'), created_at=datetime.datetime(2023, 9, 9, 12, 27, 55, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2023, 9, 9, 12, 27, 55, tzinfo=TzInfo(UTC)), parent_id=ParentId(id=ObjectId('64eca312f6197fd20d762cf5'), type='entry'), mbay='gà àlmbétɨ̀', english=Translation(translation='-light a match', key='l'), french=Translation(translation='allumer une allumette', key='a'), sound_filename=None), Example(id=ObjectId('64eca312f6197fd20d760970'), created_at=datetime.datetime(2023, 9, 9, 12, 27, 55, tzinfo=TzInfo(UTC)), up

In [55]:
selection = []
for entry in entries:
    if entry.related_word is not None and entry.grammatical_note is None:
        selection.append(entry)


print(len(selection))
rprint(selection[1])

74


Entry(
    id=ObjectId('64fc50f6286b18ef7de0a2ca'),
    created_at=datetime.datetime(2023, 9, 9, 11, 3, 18, tzinfo=TzInfo(UTC)),
    updated_at=datetime.datetime(2023, 9, 9, 11, 3, 18, tzinfo=TzInfo(UTC)),
    headword='kírì',
    part_of_speech='NI',
    sound_filename='RCN_123_20.mp3',
    french=Translation(translation='jeu joué avec des coquilles de cacahuètes {similaire à ^^}', key='j'),
    english=Translation(translation='game played with peanut shells {similar to ^^}', key='g'),
    related_word=RelatedWord(text=None, id=ObjectId('64fc50f6286b18ef7de0a235')),
    grammatical_note=None,
    examples=[
        Example(
            id=ObjectId('64fc50f6286b18ef7de0a2cb'),
            created_at=datetime.datetime(2023, 9, 9, 11, 3, 18, tzinfo=TzInfo(UTC)),
            updated_at=datetime.datetime(2023, 9, 9, 11, 3, 18, tzinfo=TzInfo(UTC)),
            parent_id=ParentId(id=ObjectId('64fc50f6286b18ef7de0a2ca'), type='entry'),
            mbay='à ɗɔ̄ɔ̄-n̄ kírì kɨ̀ ngɔ́ɔ́ wúl̄.',
            english=Translation(translation='____ is a game played with peanut shells.', key='MISC'),
            french=Translation(translation='____ est un jeu joué avec des coquilles de cacahuètes.', key='MISC'),
            sound_filename='RCJ5_29_218.mp3'
        )
    ],
    expressions=[]
)

## Data cleansing


In [48]:
entries[0]

Entry(id=ObjectId('64eca312f6197fd20d762cf5'), created_at=datetime.datetime(2023, 9, 9, 12, 27, 55, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2023, 9, 9, 12, 27, 55, tzinfo=TzInfo(UTC)), headword='àlmbétɨ̀, àlmétɨ̀', part_of_speech='NI', sound_filename='NewExpSS2215.mp3', french=Translation(translation='match', key='m'), english=Translation(translation='match.', key='m'), related_word=None, grammatical_note=None, examples=[Example(id=ObjectId('64eca312f6197fd20d76096f'), created_at=datetime.datetime(2023, 9, 9, 12, 27, 55, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2023, 9, 9, 12, 27, 55, tzinfo=TzInfo(UTC)), parent_id=ParentId(id=ObjectId('64eca312f6197fd20d762cf5'), type='entry'), mbay='gà àlmbétɨ̀', english=Translation(translation='-light a match', key='l'), french=Translation(translation='allumer une allumette', key='a'), sound_filename=None), Example(id=ObjectId('64eca312f6197fd20d760970'), created_at=datetime.datetime(2023, 9, 9, 12, 27, 55, tzinfo=TzInfo(UTC)), up

## Huggingface datasets creation


In [58]:
from typing import Literal, TypedDict


class Record(TypedDict):
    type: Literal["entry", "example", "expression"]
    mbay: str
    french: str
    english: str

In [73]:
"/-litten znjllfz ".strip("-/ ")

'litten znjllfz'

In [93]:
str.replace?

Signature: str.replace(self, old, new, count=-1, /)
Docstring:
Return a copy with all occurrences of substring old replaced by new.

  count
    Maximum number of occurrences to replace.
    -1 (the default value) means replace all occurrences.

If the optional argument count is given, only the first count occurrences are
replaced.
Type:      method_descriptor

In [103]:
"^^" in "àngɨ̀répɨ̀ is a stick bed made from ^^ wood."

True

In [112]:
def replace_related_word(txt: str, related_word: str | None = None) -> str:
    if related_word is None:
        return txt
    return txt.replace("^^", related_word)


def clean_text(txt: str, headword: str, related_word: str | None = None) -> str:
    txt = txt.replace("____", headword)
    txt = replace_related_word(txt, related_word)
    txt = txt.strip().strip("-")
    return txt


def get_related_word(entry):
    related_word: str | None = None
    if entry.related_word:
        related_word = entry.related_word.text
        if related_word is None:
            related_word = entry_map[entry.related_word.id].headword
    return related_word


def entry_to_records(entry: d.Entry, uniques: set[str]) -> Generator[Record, Any, None]:
    related_word = get_related_word(entry)

    yield {
        "id": str(entry.id),
        "entry_id": None,
        "type": "entry",
        "mbay": entry.headword,
        "french": replace_related_word(entry.french.translation, related_word),
        "english": replace_related_word(entry.english.translation, related_word),
        "edited": False,
    }

    for example in entry.examples:
        item = {
            "id": str(example.id),
            "entry_id": str(entry.id),
            "type": "example",
            "mbay": example.mbay,
            "french": clean_text(
                example.french.translation, entry.headword, related_word
            ),
            "english": clean_text(
                example.english.translation, entry.headword, related_word
            ),
            "edited": False,
        }

        dup = item["english"].lower()
        if dup in uniques:
            continue

        # Skip examples with unmatched related words. Let's be conservative about it.
        if "^^" in item["english"]:
            continue

        uniques.add(dup)
        yield item

    for expression in entry.expressions:
        item = {
            "id": str(expression.id),
            "entry_id": str(entry.id),
            "type": "expression",
            "mbay": expression.mbay,
            "french": clean_text(
                expression.french.translation, entry.headword, related_word
            ),
            "english": clean_text(
                expression.english.translation, entry.headword, related_word
            ),
            "edited": False,
        }

        dup = item["english"].lower()
        if dup in uniques:
            continue

        # Skip examples with unmatched related words. Let's be conservative about it.
        if "^^" in item["english"]:
            continue

        uniques.add(dup)
        yield item

In [113]:
uniques = set()
list(entry_to_records(selection[1], uniques))

[{'id': '64fc50f6286b18ef7de0a2ca',
  'entry_id': None,
  'type': 'entry',
  'mbay': 'kírì',
  'french': 'jeu joué avec des coquilles de cacahuètes {similaire à mbàrà}',
  'english': 'game played with peanut shells {similar to mbàrà}',
  'edited': False},
 {'id': '64fc50f6286b18ef7de0a2cb',
  'entry_id': '64fc50f6286b18ef7de0a2ca',
  'type': 'example',
  'mbay': 'à ɗɔ̄ɔ̄-n̄ kírì kɨ̀ ngɔ́ɔ́ wúl̄.',
  'french': 'kírì est un jeu joué avec des coquilles de cacahuètes.',
  'english': 'kírì is a game played with peanut shells.',
  'edited': False}]

In [114]:
uniques: set[str] = set()

records: list[Record] = []
for entry in entries:
    records.extend(entry_to_records(entry, uniques))

# Let's check the first few records
records[:5]

[{'id': '64eca312f6197fd20d762cf5',
  'entry_id': None,
  'type': 'entry',
  'mbay': 'àlmbétɨ̀, àlmétɨ̀',
  'french': 'match',
  'english': 'match.',
  'edited': False},
 {'id': '64eca312f6197fd20d76096f',
  'entry_id': '64eca312f6197fd20d762cf5',
  'type': 'example',
  'mbay': 'gà àlmbétɨ̀',
  'french': 'allumer une allumette',
  'english': 'light a match',
  'edited': False},
 {'id': '64eca312f6197fd20d760970',
  'entry_id': '64eca312f6197fd20d762cf5',
  'type': 'example',
  'mbay': 'ī-gá àlmbétɨ̀ ādɨ̄-m̄.',
  'french': 'Allumez une allumette pour moi.',
  'english': 'Light a match for me.',
  'edited': False},
 {'id': '64eca312f6197fd20d760971',
  'entry_id': '64eca312f6197fd20d762cf5',
  'type': 'example',
  'mbay': 'kùm-àlmbétɨ̀',
  'french': 'allumette non allumée',
  'english': 'unlit match stick',
  'edited': False},
 {'id': '64eca312f6197fd20d760972',
  'entry_id': '64eca312f6197fd20d762cf5',
  'type': 'example',
  'mbay': 'Màn̄ à ɔ̀dɨ̀ kùm-àlmbétɨ̀ ànḛ̄ à ùnjɨ̄ àĺ.',
  'frenc

In [115]:
len(records) / len(entries)

2.0406921467959687

In [116]:
CSV_REVIEW_DATASET_PATH = "../../datasets/mbay-translations-review.csv"
CSV_DATASET_PATH = "../../datasets/mbay-translations-flattened.csv.gzip"
SPLIT_DATASET_PATH = "../../datasets/mbay-translations/"
TOKENIZED_DATASET_PATH = "../../datasets/mbay-translations-tokenized/"

In [117]:
df = pd.DataFrame(records)
df.to_csv(CSV_REVIEW_DATASET_PATH, index=False)

We are currently at row 72



In [118]:
df = pd.read_csv(CSV_REVIEW_DATASET_PATH)

In [119]:
df

,id,entry_id,type,mbay,french,english,edited
0,64eca312f6197fd20d762cf5,NaN,entry,"àlmbétɨ̀, àlmétɨ̀",match,match.,False
1,64eca312f6197fd20d76096f,64eca312f6197fd20d762cf5,example,gà àlmbétɨ̀,allumer une allumette,light a match,False
2,64eca312f6197fd20d760970,64eca312f6197fd20d762cf5,example,ī-gá àlmbétɨ̀ ādɨ̄-m̄.,Allumez une allumette pour moi.,Light a match for me.,False
3,64eca312f6197fd20d760971,64eca312f6197fd20d762cf5,example,kùm-àlmbétɨ̀,allumette non allumée,unlit match stick,False
4,64eca312f6197fd20d760972,64eca312f6197fd20d762cf5,example,Màn̄ à ɔ̀dɨ̀ kùm-àlmbétɨ̀ ànḛ̄ à ùnjɨ̄ àĺ.,"Si l'eau touche une allumette, elle ne s'allum...",If water touches a matchstick it won't light.,False
...,...,...,...,...,...,...,...
10727,64fc50f7286b18ef7de0f25a,64fc50f7286b18ef7de0f259,example,m̄-hɔ́-ī àbàsúl kɨ̀tɔ̄-kàdɨ̄ ī-sá jī-í kɨ́rā àĺ.,"Je t'ai battu à plate couture, car tu n'as pas...","I skunked you, since you didn't take a single ...",False
10728,64fc50f7286b18ef7de0f25b,NaN,entry,à,ensuite; et donc; et ensuite,then; and so; and then,False
10729,64fc50f7286b18ef7de0f25c,64fc50f7286b18ef7de0f25b,example,Dèē-kɨ́-dḛ̀ḛ́ nā̰a̰ tá̰a̰ nèl̄ tà-á péléng à n...,La femme a goûté la sauce et elle l'a tellemen...,The woman tasted the sauce and she liked it so...,False
10730,64fc50f7286b18ef7de0f25d,NaN,entry,àbɨ̀lày,fouet fait de cuir {traditionnellement en peau...,whip made of leather {traditionally hippopotam...,False


In [14]:
df = pd.DataFrame(records)
df.to_csv(CSV_DATASET_PATH, index=False, compression="gzip")

NameError: name 'records' is not defined

In [5]:
dst = Dataset.from_csv(CSV_DATASET_PATH)
dst

Dataset({
    features: ['type', 'mbay', 'french', 'english'],
    num_rows: 11015
})

In [6]:
dst[10]

{'type': 'example',
 'mbay': 'ī-ɗāa àngérì nà̰ wétɨ́ ī-sō hólēe tɨ́ nò.',
 'french': 'Faites attention de ne pas tomber dans le trou.',
 'english': 'Be careful lest you fall into the hole.'}

In [7]:
train_test = dst.train_test_split(0.2)
test_valid = train_test["test"].train_test_split(0.5)

train_test_valid_dst = DatasetDict(
    {
        "train": train_test["train"],
        "test": test_valid["test"],
        "validation": test_valid["train"],
    }
)

In [30]:
train_test_valid_dst["test"][2]

{'type': 'example',
 'mbay': 'Ngɔ̀r làā làmíǹ gásɨ̀ ngá̰y.',
 'french': 'Récemment, les citrons sont très difficiles à trouver.',
 'english': 'Recently lemons are very hard to find.'}

In [22]:
train_test_valid_dst.save_to_disk(SPLIT_DATASET_PATH)

Saving the dataset (0/1 shards):   0%|          | 0/8812 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1102 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1101 [00:00<?, ? examples/s]

In [10]:
# T5Tokenizer.__call__?

In [31]:
from mbay_nmt.fine_tune_t5.utils import preprocess_records

In [32]:
from transformers import AutoTokenizer

t5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")

In [33]:
train_test_valid_dst.column_names["train"]

['type', 'mbay', 'french', 'english']

In [24]:
test_tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")

/Users/nasoungadoy/Code/mbay-translator/py/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [28]:
input_txt = (
    f"Translate French to Mbay: Récemment, les citrons sont très difficiles à trouver."
)
target_txt = "Ngɔ̀r làā làmíǹ gásɨ̀ ngá̰y."

In [25]:
v = test_tokenizer(
    input_txt,
    text_target=target_txt,
    max_length=512,
    truncation=True,
    padding="max_length",
)


print(v)

{'input_ids': [89349, 21273, 288, 352, 13921, 267, 8661, 297, 53675, 261, 520, 96910, 263, 259, 2759, 259, 7711, 263, 259, 31268, 299, 259, 369, 13417, 295, 260, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [26]:
print(
    sum(1 for i in v["input_ids"] if i != 0),
    sum(1 for i in v["labels"] if i != 0),
)

27 18


In [27]:
test_tokenizer.decode(v["labels"])

'Ngɔ̀r làā làmíǹ gásɨ̀ ngá̰y.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [34]:
from functools import partial

final_dst = train_test_valid_dst.map(
    partial(preprocess_records, t5_tokenizer),
    batched=True,
    remove_columns=train_test_valid_dst["train"].column_names,
)

Map:   0%|          | 0/8812 [00:00<?, ? examples/s]

Map:   0%|          | 0/1102 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

In [30]:
tokenizer.pad_token_id

0

In [ ]:
        if padding == "max_length" and data_args.ignore_pad_token_for_loss:
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

In [33]:
rprint(train_test_valid_dst["train"][0])

{
    'type': 'entry',
    'mbay': 'ngè-ndìgɨ̀-pāa',
    'french': 'personne qui chante le refrain dans les chansons',
    'english': 'person who sings refrain in songs'
}

In [29]:
# rprint(final_dst["train"][0])

In [39]:
# t5_tokenizer.decode(final_dst["train"][0]["labels"])

In [40]:
final_dst.save_to_disk(TOKENIZED_DATASET_PATH)

Saving the dataset (0/1 shards):   0%|          | 0/35248 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4408 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4404 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)